<a href="https://colab.research.google.com/github/mihaelacoste/PHD-Natural-Disasters/blob/main/Model_Scene_Composition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install tensorflow scikit-learn


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


In [ ]:
!ls '/content/drive/'


In [ ]:
!ls '/content/drive/MyDrive/'


***Model for Composition elements/scene classification***

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data directories
base_dir = '/content/drive/MyDrive/scene_classification'

# Use ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Create training data generator
train_generator = train_datagen.flow_from_directory(base_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')


Found 9497 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Build a simple CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax')) ####add here the number of classes identified at the previous step

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_generator, epochs=25)

***Vizualise Training Results***

In [ ]:
print(history.history.keys())


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

epochs_range = range(10)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')
plt.show()


In [ ]:
# Save the model
model.save("/content/model/model_scene_v2.h5")




---


**Categorizing images**

In [ ]:
import os
import shutil
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

def load_and_preprocess_image(img_path):
    # Load the image from file
    img = image.load_img(img_path, target_size=(224, 224))

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Expand the dimensions to match the input shape of the model
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the image
    img_array = preprocess_input(img_array)

    return img_array

def categorize_images(image_folder, output_folder, class_mapping):
    for image_file in os.listdir(image_folder):
        img_path = os.path.join(image_folder, image_file)
        img = load_and_preprocess_image(img_path)
        img_array = img.reshape(1, 224, 224, 3)
        prediction = model.predict(img_array)
        category = np.argmax(prediction)

        # Use class_mapping to get the actual class label
        class_label = class_mapping[category]

        # Create output folder if it doesn't exist
        category_folder = os.path.join(output_folder, f'category_{class_label}')
        os.makedirs(category_folder, exist_ok=True)

        # Move the image to the appropriate folder
        shutil.copy(img_path, os.path.join(category_folder, image_file))

# Define image and output folders
image_folder = "/content/earthquake"  # Change this to your  dataset folder
output_folder = "/content/earthquake_composition_categorization"

# Load the trained model
model = load_model("/content/model_composition_elements.h5")

# Define class mapping based on the order of classes in your training data
class_mapping = {0: 'center', 1: 'curved', 2: 'diagonal', 3: 'fill_the_frame', 4: 'golden_ratio', 5: 'horizontal', 6: 'none', 7: 'pattern', 8: 'radial', 9: 'rule_of_thirds', 10: 'symmetric', 11: 'triangle', 12: 'vanishing_point', 13: 'vertical'}

# Run categorization
categorize_images(image_folder, output_folder, class_mapping)


In [ ]:
import shutil

def zip_and_download_folder(folder_path, zip_filename):
    shutil.make_archive(zip_filename, 'zip', folder_path)

# Specify the path of the folder you want to download
output_folder = "/content/earthquake_composition_categorization"

# Specify the desired name for the zip file
zip_filename = "/content/earthquake_composition_categorization"

# Create a zip file of the folder
zip_and_download_folder(output_folder, zip_filename)


In [ ]:
from google.colab import files

# Download the zip file
files.download(zip_filename)
